In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import numpy as np

d:\Anaconda\envs\Basic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1、加载并预处理文本

In [5]:
def load_and_prepare_dataset(tokenizer, dataset_path="./imdb", split="train"):
    """
    加载并预处理文本数据集
    
    参数:
        tokenizer: 分词器，用于将文本转换为模型可理解的token
        dataset_path: 数据集名称或本地路径，这里使用IMDB电影评论数据集作为示例
        split: 要加载的数据集分割部分（train/validation/test）
    
    返回:
        预处理后的tokenized数据集
    """
    # 加载数据集 - 这里使用IMDB情感分析数据集作为示例
    # 文件格式转换为json
    parquet_path = "train-00000-of-00001.parquet"
    df = pd.read_parquet(parquet_path, engine="pyarrow")
    out_json_path = "train-00000-of-00001.json"

    df.to_json(
        out_json_path,
        orient="records", # 每行一个json对象
        force_ascii=False, # 保持中文字符
        lines=True
    )

    # 加载数据集        
    dataset = load_dataset(
        "json",
        data_files={
            "train": "train-00000-of-00001.json",
        },  split=split
    )
    print(f"成功加载数据集，共包含 {len(dataset)} 条样本")

    # 定义预处理函数
    def preprocess_function(examples):
        """
        对单批次数据进行预处理：
        1. 将文本转换为token
        2. 统一序列长度（padding和truncation）
        3. 为语言模型任务创建标签
        """
        # 对于文本生成任务，我们将使用"text"字段
        # 为每条文本添加结束标记，让模型知道生成何时结束
        texts = [f"评论: {text}\n回复: " + tokenizer.eos_token for text in examples["text"]]
        
        # 分词处理
        tokenized = tokenizer(
            texts,
            padding="max_length",  # 填充到最大长度
            max_length=128,        # 小模型适合较短的序列长度
            truncation=True,       # 截断  过长的文本
            return_tensors="pt"    # 返回PyTorch张量
        )
        
        # 对于因果语言模型（如GPT），标签与输入ID相同,所以直接克隆，因为本次的输出会作为下一层的输入
        # 因为模型的任务是预测下一个token
        tokenized["labels"] = tokenized["input_ids"].clone()
        
        return tokenized
    
    # 应用预处理函数到整个数据集
    # batched=True表示批量处理，加快速度
    tokenized_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names,  # 移除原始文本列，只保留处理后的特征
        desc="正在预处理数据集..."
    )
    
    return tokenized_dataset


### 2、加载基础模型并配置LoRA适配器

In [6]:
def setup_model_and_lora(local_model_dir="./gpt2", use_quantization=True):
    """
    加载基础模型并配置LoRA适配器
    
    参数:
        model_name: 预训练模型名称，这里使用小参数的gpt2
        use_quantization: 是否使用量化来减少显存占用
    
    返回:
        model: 配置了LoRA的模型
        tokenizer: 对应的分词器
    """
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained(local_model_dir)
    
    # GPT2默认没有pad_token，需要手动设置
    # 这里我们使用eos_token作为pad_token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 配置量化参数（如果启用）
    quantization_config = None
    if use_quantization:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,                # 启用4位量化
            bnb_4bit_quant_type="nf4",        # 使用nf4量化类型（适合自然语言）
            bnb_4bit_compute_dtype=torch.float16,  # 计算时使用float16
            bnb_4bit_use_double_quant=True    # 启用双重量化，进一步减少内存占用
        )
    
    # 加载基础模型
    model = AutoModelForCausalLM.from_pretrained(
        local_model_dir,
        quantization_config=quantization_config,
        device_map="auto",  # 自动将模型分配到可用设备
        torch_dtype=torch.float16 if use_quantization else None,
        local_files_only=True  # 强制只加载本地文件，不尝试在线下载（可选，推荐加）
    )
    
    # 如果使用量化，需要准备模型进行kbit训练
    if use_quantization:
        model = prepare_model_for_kbit_training(model)
    
    # 配置LoRA参数
    lora_config = LoraConfig(
        r=8,                      # LoRA的秩，控制低秩矩阵的维度
        lora_alpha=32,            # LoRA的缩放因子
        target_modules=["c_attn"],# 目标模块，GPT2中注意力模块的名称是c_attn
        lora_dropout=0.05,        # Dropout概率，防止过拟合
        bias="none",              # 不微调偏置参数
        task_type="CAUSAL_LM",    # 任务类型：因果语言模型
    )
    
    # 将LoRA应用到模型
    model = get_peft_model(model, lora_config)
    
    # 打印可训练参数信息
    model.print_trainable_parameters()
    
    return model, tokenizer



### 3、配置训练参数

In [7]:
def configure_training_args(output_dir="./lora_gpt_results"):
    """
    配置训练参数
    
    参数:
        output_dir: 模型和日志的保存目录
    
    返回:
        TrainingArguments对象，包含所有训练参数
    """
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)
    
    training_args = TrainingArguments(
        output_dir=output_dir,                # 输出目录
        per_device_train_batch_size=64,        # 每个设备的训练批次大小
        gradient_accumulation_steps=2,        # 梯度累积步数
        learning_rate=2e-4,                   # 学习率，LoRA通常使用比全量微调更大的学习率
        num_train_epochs=3,                   # 训练轮数
        logging_dir=f"{output_dir}/logs",     # 日志目录
        logging_steps=10,                     # 每10步打印一次日志
        save_steps=100,                       # 每100步保存一次模型
        save_total_limit=2,                   # 最多保存2个模型检查点
        fp16=True,                            # 启用混合精度训练
        report_to="tensorboard",              # 报告日志到TensorBoard
        remove_unused_columns=False,          # 不移除未使用的列
        optim="paged_adamw_8bit"              # 使用8位优化器，节省内存
    )
    
    return training_args



### 4、训练模型

In [8]:
def train_model(model, tokenized_dataset, training_args, tokenizer):
    """
    训练模型
    
    参数:
        model: 要训练的模型
        tokenized_dataset: 预处理后的训练数据集
        training_args: 训练参数配置
    
    返回:
        训练好的Trainer对象
    """
    # 数据收集器：用于将多个样本组合成批次
    # 对于语言模型，我们使用DataCollatorForLanguageModeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,  # 对于GPT等因果语言模型，不使用掩码语言模型
    )
    
    # 创建Trainer对象
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
    )
    
    # 开始训练
    print("开始训练...")
    trainer.train()
    
    # 保存最终模型
    trainer.save_model(f"{training_args.output_dir}/final_model")
    print(f"模型已保存到 {training_args.output_dir}/final_model")
    
    return trainer



### 5、生成文本函数

In [9]:

def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7):
    """
    使用训练好的模型生成文本
    
    参数:
        model: 训练好的模型
        tokenizer: 分词器
        prompt: 生成文本的提示词
        max_length: 生成文本的最大长度
        temperature: 控制生成文本的随机性（值越小越确定）
    
    返回:
        生成的文本
    """
    # 对提示词进行编码
    inputs = tokenizer(
        prompt,
        return_tensors="pt", # 返回PyTorch张量
        truncation=True, 
        max_length=512
    ).to(model.device) 
    
    # 生成文本
    with torch.no_grad():  # 不计算梯度，节省内存
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,  # 启用采样
            top_p=0.9,       # 核采样参数
            repetition_penalty=1.2,  # 重复惩罚，减少重复生成
            pad_token_id=tokenizer.pad_token_id,  # 
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 解码生成的token为文本
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text



### 6、训练流程

In [10]:

if __name__ == "__main__":
    # 1. 加载分词器（先临时加载用于处理数据）
    temp_tokenizer = AutoTokenizer.from_pretrained("./gpt2")
    if temp_tokenizer.pad_token is None:
        temp_tokenizer.pad_token = temp_tokenizer.eos_token
    
    # 2. 加载和预处理数据集
    tokenized_dataset = load_and_prepare_dataset(temp_tokenizer)
    
    # 3. 设置模型和LoRA
    model, tokenizer = setup_model_and_lora(
        local_model_dir="./gpt2",
        use_quantization=False  # 小模型也可以使用量化进一步节省内存
    )
    
    # 4. 配置训练参数
    training_args = configure_training_args()
    
    # 5. 训练模型（2、3、4）
    trainer = train_model(model, tokenized_dataset, training_args,tokenizer)
    


Generating train split: 25000 examples [00:00, 271083.66 examples/s]


成功加载数据集，共包含 25000 条样本


正在预处理数据集...: 100%|██████████| 25000/25000 [00:05<00:00, 4203.95 examples/s]
The 8-bit optimizer is not available on your device, only available on CUDA for now.
Exception in thread Thread-5:
Traceback (most recent call last):
  File "d:\Anaconda\envs\Basic\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "d:\Anaconda\envs\Basic\lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "d:\Anaconda\envs\Basic\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Anaconda\envs\Basic\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "d:\Anaconda\envs\Basic\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb2 in position 7: invalid start byte
d:\Anaconda\envs\Basic\lib\site-packages\peft\tuners\lora\layer.py:2174: UserWarning: fan_in_fan_ou

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364
开始训练...


d:\Anaconda\envs\Basic\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


KeyboardInterrupt: 

## 7、生成文本测试

In [ ]:
# 6. 测试文本生成
test_prompts = [
    "评论: 这部电影真的很好看！特别推荐\n回复: ",
    "评论: 这是我看过的最糟糕的电影《战狼》，浪费时间。\n回复: "
]

print("\n===== 文本生成测试 =====")
for prompt in test_prompts:
    generated = generate_text(model, tokenizer, prompt, max_length=150)
    for item in generated.split("\n"):
        # 先处理字符串内部的 "."，再打印（自动换行）
        processed_item = item.replace(".", ".\n")
        print(processed_item)
